## Import Libraries for preprocessing

In [15]:
import re
import requests
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import jieba
from sklearn.model_selection import train_test_split

## Preprocessing

```
 預處理步驟：
 1. 讀取 csv 檔案中的 item names -> 取得所有產品的 item names。
 2. 因在本機運算(使用cpu)，使用少部分資料進行模型訓練測試。
 3. 準備訓練、測試資料集 -> 因我們的資料為 unsupervised，故以jieba+預處理的結果作為斷詞之 ground_truth。
```

In [2]:
def getItems(file_name, item_col):
    df = pd.read_csv(file_name)
    return df[item_col].tolist()
    
def filterItem(item_name, stopwords, all_items):
    item_idx = [idx for idx, name in enumerate(all_items) if item_name in name.lower()]
    item_list = [all_item_names[idx].lower() for idx in item_idx]
    
    filter_item_name = []
    for item in item_list:
        item_name = []
        item = re.sub(r'[^\w]', ' ', item) # remove symbols
        item = re.sub(r"\s+", " ", item)  # remove 一個以上的空白字符
        split_item = item.strip().split(' ')
        
        for word in split_item:
            if word in stopwords:
                continue
            else:
                item_name.append(word)
        filter_item_name.append(" ".join(item_name))
    
    return filter_item_name

In [3]:
#1. get items
all_item_names = getItems('mall_all_item.csv', 'item_name')

# 2. filer items
item_name = 'iphone'
stopwords = []
filter_item_list = filterItem(item_name, stopwords, all_item_names)
print("Length of the filter items: ", len(filter_item_list))
print()
print(filter_item_list[:5])

10114

In [6]:
# 3.the function that tag the word
def words_to_tags(sent):
    tags = []
    for word in sent:
        if len(word) == 1:
            tags.append('S')
        else:
            for i in range(len(word)):
                if i == 0:
                    tags.append('L')
                elif i == len(word) - 1:
                    tags.append('R')
                else:
                    tags.append('M')                    
    return tags

In [7]:
# 3. the function for preparing the train, test dataset
def prepareDataset(item_list):
    X = []
    y = []
    raw = []
    
    word_to_ix = {}
    word_to_ix['UNK'] = 0
    
    for item in item_list:
        split_item_sent = list(jieba.cut(item))
        raw.append(split_item_sent)
        
        sent_words = list("".join(split_item_sent))
        for word in sent_words:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
        
        X.append(sent_words)
        y.append(words_to_tags(split_item_sent))
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                            test_size=0.2, random_state=42)
    
    raw_train, raw_test = train_test_split(raw, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, raw_train, raw_test, word_to_ix

X_train, X_test, y_train, y_test, raw_train, raw_test, word_to_ix = prepareDataset(filter_item_list)

------------------------------------------------------

## Import Libraries for building model with Pytorch

```
 1. import torch 相關模組，並設定建構模型的 hypterparameters。
 2. 此處的 Bi-Lstm crf model 是參照 pytorch 官網的 tutorial 所建構，並稍作調整，tutorial 連結附註在底處。
    後續再貼上此 pytorch 模型架構的 code 筆記。
 3. 在此處，我並沒有使用到 viterbi algorithm 去尋找序列的最佳路徑，而是透過 neg_log_likelihood 計算模型損失，
 　 再用 backpropagation 去更新參數優化。
```

> [Bi-Lstm crf pytorch-tutorial](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

torch.manual_seed(1)

In [9]:
# 1. set the hyper-parameters
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 16
HIDDEN_DIM = 16

tag_to_ix = {"L": 0, "M": 1, "R": 2, "S": 3, START_TAG: 4, STOP_TAG: 5}

In [10]:
# 2. function for building model

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = []
    for word in seq:
        if word in to_ix:
            ix = to_ix[word]
            idxs.append(ix)
        else:
            ix = to_ix['UNK']
            idxs.append(ix) 
            
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [11]:
# 2. class of the architecture of BiLSTM_CRF model

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.
        # Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        feats = feats.to(device)
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas.to(device)

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size).to(device)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1).to(device)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        embeds = embeds.to(device)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        feats = feats.to(device)
        tags = tags.to(device)
        
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        temp = torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device)
        tags = torch.cat([temp, tags])
        
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        feats = feats.to(device)
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars.to(device)
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        sentence = sentence.to(device)
        tags = tags.to(device)
        
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        setence = sentence.to(device)
        
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [12]:
# set the device, model and optimizer\
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
print(device)

cpu


---

## Before training our Bi-LSTM CRF model

```
 由以下結果，可發現模型在還沒 training 之前預測結果非常差。
```

In [13]:
with torch.no_grad():
    precheck_sent = prepare_sequence(X_train[0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in y_train[0]], dtype=torch.long)
    print(precheck_tags)
    print()
    print("Word tag before training: ", model(precheck_sent)[1])

tensor([0, 1, 1, 1, 2, 3, 0, 1, 1, 1, 1, 1, 2, 3, 0, 1, 1, 1, 1, 2, 3, 3, 3, 3,
        3, 0, 1, 1, 2, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 2])

Word tag before training:  [2, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3]


## Training our Bi-LSTM CRF model

In [14]:
from tqdm import tqdm_notebook
length = len(X_train)

for epoch in tqdm_notebook(range(5)):
    for i in tqdm_notebook(range(length)):
        sentence = X_train[i]
        tags = y_train[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

---

## Evaluation Part

```
  訓練完模型後，就可透過 model 去 predict 出序列的標註結果。
  1. segment() -> 用來將 model 預測出之序列標註作為特徵，並把 sentence 斷成 tokens。
  　 斷詞標準：當 tag 出現 0("L"->代表 tokens 最左邊) 或 3("S"->代表單字成詞)
    
  2. 轉換過後即可計算 model prediction 與 ground truth 的差別，並計算 recall, precision, f1 score。
```

In [16]:
def segment(sent, word_to_ix, model):
    with torch.no_grad():
        precheck_sent = prepare_sequence(sent, word_to_ix)
        precheck_sent = precheck_sent.to(device)

        tags = model(precheck_sent)[1]
    
    tokens = []
    tok = ""
    for ch, tag in zip(list(sent), tags):
        # 0 tag -> "L", 3 tag -> "S"
        if tag in [0, 3] and tok != "":
            tokens.append(tok)
            tok = ""
        tok += ch
    if tok:
        tokens.append(tok)
        
    return tokens

In [17]:
# 2. Function for calculating the difference between actual and predicted ones
def compare(actual_toks, pred_toks):
    i = 0
    j = 0
    p = 0
    q = 0
    tp = 0
    fp = 0
    while i < len(actual_toks) and j < len(pred_toks):
        if p == q:
            if actual_toks[i] == pred_toks[j]:
                tp += 1
            else:
                fp += 1
            p += len(actual_toks[i])
            q += len(pred_toks[j])
            i += 1
            j += 1
        elif p < q:
            p += len(actual_toks[i])
            i += 1
        else:
            fp += 1
            q += len(pred_toks[j])
            j += 1
    return tp, fp, len(actual_toks)


# 2. Function for calculating recall, precision, f1 score
def score(actual_sents, pred_sents):
    tp = 0
    fp = 0
    total = 0
    for actual_toks, pred_toks in zip(actual_sents, pred_sents):
        tp_, fp_, total_ = compare(actual_toks, pred_toks)
        tp += tp_
        fp += fp_
        total += total_
    recall = float(tp) / total
    precision = float(tp) / (tp + fp)
    f1 = 2.0 * recall * precision / (recall + precision)
    return recall, precision, f1

In [18]:
pred = []
actual = []
for sent in tqdm_notebook(raw_test):
    pred.append(segment("".join(sent), word_to_ix, model))
    actual.append(sent)

recall, precision, f1_score = score(actual, pred)

print("Recall: ", recall)
print("Precision: ", precision)
print("F1 score: ", f1_score)


Recall:  0.9677996062833956
Precision:  0.9722917339360672
F1 score:  0.9700404695270501
